In [6]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm import tqdm
import pandas as pd



In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
path = "/content/drive/MyDrive/scom/admin.csv"
df = pd.read_csv(path)


<ipython-input-5-a437909a3357>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
<ipython-input-5-a437909a3357>:6: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [16]:

label_mapping = {'normal': 0, 'toxic': 1, 'sexism': 2, 'racism': 3}
df['label'] = df['label'].map(label_mapping)


train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [17]:

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_encodings = tokenizer(list(train_df['text']), truncation=True, padding=True, max_length=128, return_tensors='pt')
test_encodings = tokenizer(list(test_df['text']), truncation=True, padding=True, max_length=128, return_tensors='pt')

In [18]:

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(list(train_df['label'])))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(list(test_df['label'])))


In [ ]:

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(label_mapping))
optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataset))


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)


In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        inputs = {'input_ids': batch[0].to(device), 'attention_mask': batch[1].to(device), 'labels': batch[2].to(device)}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

    average_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss}')


Epoch 1/5: 100%|██████████| 100/100 [20:38<00:00, 12.38s/it]


Epoch 1/5, Average Loss: 1.0213697849214076


Epoch 2/5: 100%|██████████| 100/100 [19:48<00:00, 11.89s/it]


Epoch 2/5, Average Loss: 0.4888869158178568


Epoch 3/5: 100%|██████████| 100/100 [19:45<00:00, 11.85s/it]


Epoch 3/5, Average Loss: 0.2896485515683889


Epoch 4/5: 100%|██████████| 100/100 [19:36<00:00, 11.77s/it]


Epoch 4/5, Average Loss: 0.1622793285548687


Epoch 5/5: 100%|██████████| 100/100 [19:44<00:00, 11.85s/it]

Epoch 5/5, Average Loss: 0.08690070183947682


In [ ]:
# Evaluate on the test set
model.eval()
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc='Evaluating'):
        inputs = {'input_ids': batch[0].to(device), 'attention_mask': batch[1].to(device)}
        labels = batch[2].cpu().numpy()
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels)


Evaluating: 100%|██████████| 25/25 [01:29<00:00,  3.60s/it]


In [ ]:
accuracy = accuracy_score(all_labels, all_preds)
classification_rep = classification_report(all_labels, all_preds, target_names=label_mapping.keys())
conf_matrix = confusion_matrix(all_labels, all_preds)

print(f'\nAccuracy: {accuracy}')
print('\nClassification Report:')
print(classification_rep)
print('\nConfusion Matrix:')
print(conf_matrix)



Accuracy: 0.765

Classification Report:
              precision    recall  f1-score   support

      normal       0.76      0.48      0.59        61
       toxic       0.81      0.90      0.85        51
      sexism       0.67      0.92      0.78        49
      racism       0.87      0.85      0.86        39

    accuracy                           0.77       200
   macro avg       0.78      0.79      0.77       200
weighted avg       0.77      0.77      0.75       200


Confusion Matrix:
[[29  8 19  5]
 [ 5 46  0  0]
 [ 3  1 45  0]
 [ 1  2  3 33]]


In [ ]:
sample_indices = [0, 1, 2]
for i in sample_indices:
    text = test_df.iloc[i]['text']
    true_label = test_df.iloc[i]['label']
    predicted_label = all_preds[i]
    print(f'\nText: {text}\nTrue Label: {true_label}, Predicted Label: {predicted_label}')



Text: "XSHOGUNX'S EYE ARE SPREAD FAR AS FUCK. HE LOOKS LIKE HE HAS DOWN'S."
True Label: 1, Predicted Label: 1

Text: @DianH4 The Saudis were the Mujahedeen before there was any involvement of the US.
True Label: 3, Predicted Label: 3

Text: Omfg Kat. I will cut you for being horrible. Do. Not. #MKR  #mykitchenrules
True Label: 0, Predicted Label: 2
